<a href="https://colab.research.google.com/github/HarshithaShetty27/CerebralFusion/blob/main/Brain_tumor_hybrid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import zipfile
zip_path = '/content/drive/MyDrive/Brain_Tumor_Dataset/brain_tumor.zip'
extract_path = '/content/Brain_Tumor_Dataset'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

base_dir = '/content/Brain_Tumor_Dataset'
categories = ['glioma', 'meningioma', 'notumor', 'pituitary']

# Split dataset into training, validation, and test sets
train_dir = os.path.join(base_dir, 'Training')
val_dir = os.path.join(base_dir, 'Validation')
test_dir = os.path.join(base_dir, 'Testing')

for category in categories:
    os.makedirs(os.path.join(train_dir, category), exist_ok=True)
    os.makedirs(os.path.join(val_dir, category), exist_ok=True)
    os.makedirs(os.path.join(test_dir, category), exist_ok=True)

# Use ImageDataGenerator for loading and augmentation
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.15)

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training')

val_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation')

test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

Found 4857 images belonging to 4 classes.
Found 855 images belonging to 4 classes.
Found 1311 images belonging to 4 classes.


In [ ]:
from skimage.filters import threshold_otsu
from skimage import img_as_ubyte
import tensorflow as tf

def preprocess_image(image):
    # OTSU thresholding
    thresh = threshold_otsu(image)
    binary = image > thresh
    binary = img_as_ubyte(binary)

    # Resize and normalize
    image = tf.image.resize(image, [224, 224])
    image = image / 255.0

    return image

In [ ]:
datagen_aug = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

train_generator_aug = datagen_aug.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

Found 5712 images belonging to 4 classes.


In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from google.colab import drive
import os

# Mount Google Drive
# drive.mount('/content/drive')

# Define the path to save the model in Google Drive
model_save_path = '/content/drive/MyDrive/models/vgg16_model.h5'

# Check if the model already exists
if os.path.exists(model_save_path):
    # Load the pre-trained model from Google Drive
    from tensorflow.keras.models import load_model
    model_vgg = load_model(model_save_path)
    print("Model loaded from Google Drive.")
else:
    # Build the VGG16 model
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(len(categories), activation='softmax')(x)
    model_vgg = Model(inputs=base_model.input, outputs=predictions)

    # Freeze the base model layers
    for layer in base_model.layers:
        layer.trainable = False

    # Compile the model
    model_vgg.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Train the model
    history_vgg = model_vgg.fit(train_generator_aug, validation_data=val_generator, epochs=10)

    # Save the trained model to Google Drive
    model_vgg.save(model_save_path)
    print("Model trained and saved to Google Drive.")

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 4108s 23s/step - accuracy: 0.7071 - loss: 1.6566 - val_accuracy: 0.2398 - val_loss: 1.3465
Epoch 2/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 4087s 23s/step - accuracy: 0.8834 - loss: 0.3326 - val_accuracy: 0.2316 - val_loss: 1.4051
Epoch 3/10
 54/179 ━━━━━━━━━━━━━━━━━━━━ 41:02 20s/step - accuracy: 0.8886 - loss: 0.3007

In [ ]:
from tensorflow.keras.applications import ResNet50

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(len(categories), activation='softmax')(x)
model_resnet = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model_resnet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history_resnet = model_resnet.fit(train_generator_aug, validation_data=val_generator, epochs=10)

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 115s 563ms/step - accuracy: 0.6990 - loss: 1.4003 - val_accuracy: 0.2421 - val_loss: 2.4756
Epoch 2/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 89s 499ms/step - accuracy: 0.8866 - loss: 0.2972 - val_accuracy: 0.2795 - val_loss: 3.1026
Epoch 3/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 88s 489ms/step - accuracy: 0.8898 - loss: 0.3044 - val_accuracy: 0.2795 - val_loss: 3.3237
Epoch 4/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 88s 493ms/step - accuracy: 0.9150 - loss: 0.2166 - val_accuracy: 0.2550 - val_loss: 3.2220
Epoch 5/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 88s 492ms/step - accuracy: 0.9179 - loss: 0.2087 - val_accuracy: 0.2772 - val_loss: 3.2172
Epoch 6/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 88s 489ms/step - accuracy: 0.9246 - loss: 0.1936 - val_accuracy: 0.2035 - val_loss: 3.2005
Epoch 7/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 87s 488ms/step - accuracy: 0.9266 - loss: 0.1951 - val_accuracy: 0.2526 - val_loss: 3.2894
Epoch 8/10
179/179 ━━━━━━━━━━━━

In [ ]:
from tensorflow.keras.applications import DenseNet121

base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(len(categories), activation='softmax')(x)
model_densenet = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model_densenet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history_densenet = model_densenet.fit(train_generator_aug, validation_data=val_generator, epochs=10)

29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 153s 690ms/step - accuracy: 0.5813 - loss: 2.3508 - val_accuracy: 0.2772 - val_loss: 1.6428
Epoch 2/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 95s 531ms/step - accuracy: 0.7823 - loss: 0.5755 - val_accuracy: 0.4690 - val_loss: 1.3644
Epoch 3/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 99s 554ms/step - accuracy: 0.7989 - loss: 0.5539 - val_accuracy: 0.3871 - val_loss: 1.4957
Epoch 4/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 90s 502ms/step - accuracy: 0.8160 - loss: 0.4843 - val_accuracy: 0.4246 - val_loss: 1.3190
Epoch 5/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 90s 500ms/step - accuracy: 0.8203 - loss: 0.4694 - val_accuracy: 0.4444 - val_loss: 1.3545
Epoch 6/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 86s 478ms/step - accuracy: 0.8245 - loss: 0.4695 - val_accuracy: 0.4199 - val_loss: 1.5599
Epoch 7/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 87s 486ms/step - accuracy: 0.8431 - loss: 0.3982 - val_accuracy: 0.4784 - val_loss: 1.3908
Epoch 8/10
179/179 ━━━━━━━━━━━━

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model_custom = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(len(categories), activation='softmax')
])

model_custom.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history_custom = model_custom.fit(train_generator_aug, validation_data=val_generator, epochs=10)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 96s 499ms/step - accuracy: 0.3806 - loss: 35.5871 - val_accuracy: 0.2316 - val_loss: 2.7371
Epoch 2/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 84s 470ms/step - accuracy: 0.5537 - loss: 0.9984 - val_accuracy: 0.2842 - val_loss: 3.3439
Epoch 3/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 86s 483ms/step - accuracy: 0.6091 - loss: 0.9163 - val_accuracy: 0.2386 - val_loss: 3.1976
Epoch 4/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 141s 477ms/step - accuracy: 0.6284 - loss: 0.8801 - val_accuracy: 0.2316 - val_loss: 4.5351
Epoch 5/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 85s 475ms/step - accuracy: 0.6745 - loss: 0.8076 - val_accuracy: 0.2316 - val_loss: 4.8935
Epoch 6/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 85s 476ms/step - accuracy: 0.7210 - loss: 0.7079 - val_accuracy: 0.2316 - val_loss: 4.8622
Epoch 7/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 88s 490ms/step - accuracy: 0.7322 - loss: 0.6931 - val_accuracy: 0.2316 - val_loss: 5.4068
Epoch 8/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 84s 471ms/step - accuracy: 0.7615 - loss:

In [ ]:
# from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, UpSampling2D, concatenate
# from tensorflow.keras.models import Model

# def unet_model(input_size=(224, 224, 3)):
#     inputs = Input(input_size)

#     # Downsample path
#     conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
#     conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv1)
#     pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

#     conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool1)
#     conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv2)
#     pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

#     conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool2)
#     conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv3)
#     pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

#     # Bottleneck
#     conv4 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool3)
#     conv4 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv4)

#     # Upsample path
#     up5 = UpSampling2D(size=(2, 2))(conv4)
#     up5 = Conv2D(256, (2, 2), activation='relu', padding='same')(up5)
#     merge5 = concatenate([conv3, up5], axis=3)
#     conv5 = Conv2D(256, (3, 3), activation='relu', padding='same')(merge5)
#     conv5 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv5)

#     up6 = UpSampling2D(size=(2, 2))(conv5)
#     up6 = Conv2D(128, (2, 2), activation='relu', padding='same')(up6)
#     merge6 = concatenate([conv2, up6], axis=3)
#     conv6 = Conv2D(128, (3, 3), activation='relu', padding='same')(merge6)
#     conv6 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv6)

#     up7 = UpSampling2D(size=(2, 2))(conv6)
#     up7 = Conv2D(64, (2, 2), activation='relu', padding='same')(up7)
#     merge7 = concatenate([conv1, up7], axis=3)
#     conv7 = Conv2D(64, (3, 3), activation='relu', padding='same')(merge7)
#     conv7 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv7)

#     # Output for segmentation
#     outputs_seg = Conv2D(1, (1, 1), activation='sigmoid')(conv7)

#     # Classification head
#     flatten = Flatten()(conv7)
#     dense1 = Dense(128, activation='relu')(flatten)
#     outputs_cls = Dense(len(categories), activation='softmax')(dense1)

#     model = Model(inputs=inputs, outputs=[outputs_seg, outputs_cls])
#     return model

# # Compile the model
# model_unet = unet_model()
# model_unet.compile(optimizer='adam',loss=['binary_crossentropy', 'categorical_crossentropy'],metrics=['accuracy'])

# # Train the model
# history_unet = model_unet.fit(train_generator_aug,validation_data=val_generator, epochs=10)


In [ ]:
import cv2
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Function to extract Gabor features
def extract_gabor_features(images):
    features = []
    kernels = []
    for theta in range(4):
        theta = theta / 4. * np.pi
        for sigma in (1, 3):
            for lamda in np.arange(0, np.pi, np.pi / 4):
                for gamma in (0.05, 0.5):
                    kernel = cv2.getGaborKernel((21, 21), sigma, theta, lamda, gamma, 0, ktype=cv2.CV_32F)
                    kernels.append(kernel)

    for image in images:
        feature_vector = []
        for kernel in kernels:
            filtered = cv2.filter2D(image, cv2.CV_8UC3, kernel)
            feature_vector.append(filtered.mean())
        features.append(feature_vector)
    return np.array(features)

# Load images and labels
images = []
labels = []
for category in categories:
    category_dir = os.path.join(train_dir, category)
    for img_name in os.listdir(category_dir):
        img_path = os.path.join(category_dir, img_name)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (224, 224))
        images.append(img)
        labels.append(categories.index(category))

# Extract Gabor features
X = extract_gabor_features(images)
y = np.array(labels)

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Train SVM
svm_model = SVC(kernel='linear', probability=True)
svm_model.fit(X_train, y_train)

# Evaluate SVM
y_pred = svm_model.predict(X_val)
print("SVM Accuracy:", accuracy_score(y_val, y_pred))

NameError: name 'categories' is not defined

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from skimage.feature import hog
from skimage import exposure
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


def extract_hog_features(images):
    features = []
    for image in images:
        # Convert RGB image to grayscale if necessary
        if len(image.shape) == 3 and image.shape[2] == 3:  # Check if image is RGB
            image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        # Extract HOG features
        fd, hog_image = hog(image, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=True, channel_axis=None)
        features.append(fd)
    return np.array(features)

# Load images and labels
images = []
labels = []
for category in categories:
    category_dir = os.path.join(train_dir, category)
    for img_name in os.listdir(category_dir):
        img_path = os.path.join(category_dir, img_name)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Load image in grayscale
        img = cv2.resize(img, (224, 224))  # Resize image
        images.append(img)
        labels.append(categories.index(category))

# Extract HOG features
X_hog = extract_hog_features(images)
y = np.array(labels)

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_hog, y, test_size=0.2, random_state=42)

# Train Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Evaluate Random Forest
y_pred = rf_model.predict(X_val)
print("Random Forest Accuracy:", accuracy_score(y_val, y_pred))
print("Random Forest Accuracy:", accuracy_score(y_val, y_pred))

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

# Function to evaluate a model
def evaluate_model(model, X_val, y_val, is_deep_learning=False):
    if is_deep_learning:
        y_pred_proba = model.predict(X_val)
        y_pred = np.argmax(y_pred_proba, axis=1)
        y_val = np.argmax(y_val, axis=1)  # Convert one-hot to integer labels
    else:
        y_pred = model.predict(X_val)
        y_pred_proba = model.predict_proba(X_val)  # For binary classification

    accuracy = accuracy_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred, average='weighted')
    # Handle AUC-ROC correctly for multi-class classification
    if len(np.unique(y_val)) > 2:  # Multi-class case
        auc_roc = roc_auc_score(y_val, y_pred_proba, multi_class='ovr')
    else:  # Binary case
        auc_roc = roc_auc_score(y_val, y_pred_proba[:, 1])  # Take probability of class 1
    return accuracy, f1, auc_roc

# Evaluate all models
models = {
    'VGG16': model_vgg,
    'ResNet50': model_resnet,
    'DenseNet': model_densenet,
    'CNN': model_custom,
    'SVM': svm_model,
    'RandomForest': rf_model
}

results = {}
# Get the full validation data
X_val_dl_full = []
y_val_dl_full = []
for i in range(len(val_generator)):
    X_batch, y_batch = val_generator[i]
    X_val_dl_full.append(X_batch)
    y_val_dl_full.append(y_batch)
X_val_dl_full = np.vstack(X_val_dl_full)
y_val_dl_full = np.vstack(y_val_dl_full)

# Convert RGB images to grayscale for HOG feature extraction
X_val_dl_gray = np.array([cv2.cvtColor(img, cv2.COLOR_RGB2GRAY) if len(img.shape) == 3 else img for img in X_val_dl_full])

# Extract features for SVM and Random Forest
X_val_gabor = extract_gabor_features(X_val_dl_full)  # Gabor features for SVM
X_val_hog = extract_hog_features(X_val_dl_gray)  # HOG features for Random Forest

# Evaluate each model
for name, model in models.items():
    if name in ['VGG16', 'ResNet50', 'DenseNet', 'CNN']:
        # For deep learning models, use the full validation data
        accuracy, f1, auc_roc = evaluate_model(model, X_val_dl_full, y_val_dl_full, is_deep_learning=True)
    else:
        # For SVM and Random Forest, use the feature-extracted validation set
        if name == 'SVM':
            X_val_fe = X_val_gabor  # Use Gabor features for SVM
        else:
            X_val_fe = X_val_hog  # Use HOG features for Random Forest
        accuracy, f1, auc_roc = evaluate_model(model, X_val_fe, np.argmax(y_val_dl_full, axis=1), is_deep_learning=False)
    results[name] = {'Accuracy': accuracy, 'F1-Score': f1, 'AUC-ROC': auc_roc}

# Convert results to a DataFrame for easy comparison
import pandas as pd
results_df = pd.DataFrame(results).T
print("Model Performance:")
print(results_df)

# Dynamically select top 3 models based on a combined score (e.g., average of normalized metrics)
results_df['Combined_Score'] = (results_df['Accuracy'] + results_df['F1-Score'] + results_df['AUC-ROC']) / 3
top_3_models = results_df.nlargest(3, 'Combined_Score').index.tolist()
print("\nTop 3 Models:",top_3_models)

In [ ]:
# Get predictions from top 3 models
y_preds = []
for model_name in top_3_models:
    model = models[model_name]
    if model_name in ['VGG16', 'ResNet50', 'DenseNet', 'CNN']:
        # For deep learning models, use the full validation data
        y_pred_proba = model.predict(X_val_dl_full)
    else:
        # For SVM and Random Forest, use the feature-extracted validation set
        if model_name == 'SVM':
            X_val_fe = X_val_gabor  # Use Gabor features for SVM
        else:
            X_val_fe = X_val_hog  # Use HOG features for Random Forest
        y_pred_proba = model.predict_proba(X_val_fe)
    y_preds.append(y_pred_proba)

In [ ]:
# Weighted averaging (weights based on model performance)
weights = results_df.loc[top_3_models, 'Combined_Score'].values
weights /= weights.sum()  # Normalize weights
y_weighted_avg = np.average(y_preds, axis=0, weights=weights)
y_weighted_avg_classes = np.argmax(y_weighted_avg, axis=1)

# Evaluate weighted averaging hybrid model
accuracy_weighted = accuracy_score(np.argmax(y_val_dl_full, axis=1), y_weighted_avg_classes)
f1_weighted = f1_score(np.argmax(y_val_dl_full, axis=1), y_weighted_avg_classes, average='weighted')
auc_roc_weighted = roc_auc_score(np.argmax(y_val_dl_full, axis=1), y_weighted_avg, multi_class='ovr')
print("\nWeighted Averaging Hybrid Model Performance:")
print(f"Accuracy: {accuracy_weighted}, F1-Score: {f1_weighted}, AUC-ROC: {auc_roc_weighted}")

In [ ]:
from sklearn.linear_model import LogisticRegression

# Stack predictions from top 3 models
X_stack = np.hstack(y_preds)

# Train a meta-learner (Logistic Regression)
meta_learner = LogisticRegression()
meta_learner.fit(X_stack, np.argmax(y_val_dl_full, axis=1))

# Evaluate stacking hybrid model
y_stack_pred = meta_learner.predict(X_stack)
y_stack_pred_proba = meta_learner.predict_proba(X_stack)

accuracy_stack = accuracy_score(np.argmax(y_val_dl_full, axis=1), y_stack_pred)
f1_stack = f1_score(np.argmax(y_val_dl_full, axis=1), y_stack_pred, average='weighted')
auc_roc_stack = roc_auc_score(np.argmax(y_val_dl_full, axis=1), y_stack_pred_proba, multi_class='ovr')
print("\nStacking Hybrid Model Performance:")
print(f"Accuracy: {accuracy_stack}, F1-Score: {f1_stack}, AUC-ROC: {auc_roc_stack}")

In [ ]:
from scipy.stats import mode

# Majority voting
y_pred_classes = np.array([np.argmax(y_pred, axis=1) for y_pred in y_preds])  # Stack predictions into a 2D array
print("Shape of y_pred_classes:", y_pred_classes.shape)  # Verify the shape

y_majority_vote, _ = mode(y_pred_classes, axis=0)  # Apply mode along the correct axis
y_majority_vote = y_majority_vote.flatten()  # Flatten the result to get 1D array

# Compute average probability predictions for AUC-ROC
y_avg_proba = np.mean(y_preds, axis=0)  # Average probabilities across models

# Verify shapes
print("Shape of y_avg_proba:", y_avg_proba.shape)
print("Shape of y_val_dl_full:", y_val_dl_full.shape)

# Evaluate majority voting hybrid model
accuracy_majority = accuracy_score(np.argmax(y_val_dl_full, axis=1), y_majority_vote)
f1_majority = f1_score(np.argmax(y_val_dl_full, axis=1), y_majority_vote, average='weighted')
auc_roc_majority = roc_auc_score(np.argmax(y_val_dl_full, axis=1), y_avg_proba, multi_class='ovr')
print("\nMajority Voting Hybrid Model Performance:")
print(f"Accuracy: {accuracy_majority}, F1-Score: {f1_majority}, AUC-ROC: {auc_roc_majority}")

In [ ]:
# Compare hybrid model results
hybrid_results = {
    'Weighted Averaging': {'Accuracy': accuracy_weighted, 'F1-Score': f1_weighted, 'AUC-ROC': auc_roc_weighted},
    'Stacking': {'Accuracy': accuracy_stack, 'F1-Score': f1_stack, 'AUC-ROC': auc_roc_stack},
    'Majority Voting': {'Accuracy': accuracy_majority, 'F1-Score': f1_majority, 'AUC-ROC': auc_roc_majority}
}

# Convert to DataFrame for easy comparison
hybrid_results_df = pd.DataFrame(hybrid_results).T
hybrid_results_df['Combined_Score'] = (hybrid_results_df['Accuracy'] + hybrid_results_df['F1-Score'] + hybrid_results_df['AUC-ROC']) / 3
print("Hybrid Model Performance:")
print(hybrid_results_df)

# Select the best hybrid model
best_hybrid_model = hybrid_results_df.idxmax()['Combined_Score']
print("\nBest Hybrid Model:", best_hybrid_model)

In [ ]:
# Get the full test data
X_test_dl_full = []
y_test_dl_full = []
for i in range(len(test_generator)):
    X_batch, y_batch = test_generator[i]
    X_test_dl_full.append(X_batch)
    y_test_dl_full.append(y_batch)
X_test_dl_full = np.vstack(X_test_dl_full)
y_test_dl_full = np.vstack(y_test_dl_full)

# Convert RGB images to grayscale for HOG feature extraction (if needed)
X_test_dl_gray = np.array([cv2.cvtColor(img, cv2.COLOR_RGB2GRAY) if len(img.shape) == 3 else img for img in X_test_dl_full])

# Extract features for SVM and Random Forest
X_test_gabor = extract_gabor_features(X_test_dl_full)  # Gabor features for SVM
X_test_hog = extract_hog_features(X_test_dl_gray)  # HOG features for Random Forest

In [ ]:
if best_hybrid_model == 'Weighted Averaging':
    # Get predictions from top 3 models for the test set
    y_preds_test = []
    for model_name in top_3_models:
        model = models[model_name]
        if model_name in ['VGG16', 'ResNet50', 'DenseNet', 'CNN']:
            y_pred_proba = model.predict(X_test_dl_full)
        else:
            if model_name == 'SVM':
                X_test_fe = X_test_gabor  # Use Gabor features for SVM
            else:
                X_test_fe = X_test_hog  # Use HOG features for Random Forest
            y_pred_proba = model.predict_proba(X_test_fe)
        y_preds_test.append(y_pred_proba)

    # Weighted averaging (weights based on model performance)
    weights = results_df.loc[top_3_models, 'Combined_Score'].values
    weights /= weights.sum()  # Normalize weights
    y_weighted_avg_test = np.average(y_preds_test, axis=0, weights=weights)
    y_weighted_avg_classes_test = np.argmax(y_weighted_avg_test, axis=1)

    # Evaluate weighted averaging hybrid model on the test set
    accuracy_test = accuracy_score(np.argmax(y_test_dl_full, axis=1), y_weighted_avg_classes_test)
    f1_test = f1_score(np.argmax(y_test_dl_full, axis=1), y_weighted_avg_classes_test, average='weighted')
    auc_roc_test = roc_auc_score(np.argmax(y_test_dl_full, axis=1), y_weighted_avg_test, multi_class='ovr')
    print("\nWeighted Averaging Hybrid Model Performance on Test Set:")
    print(f"Accuracy: {accuracy_test}, F1-Score: {f1_test}, AUC-ROC: {auc_roc_test}")

elif best_hybrid_model == 'Stacking':
    # Get predictions from top 3 models for the test set
    y_preds_test = []
    for model_name in top_3_models:
        model = models[model_name]
        if model_name in ['VGG16', 'ResNet50', 'DenseNet', 'CNN']:
            y_pred_proba = model.predict(X_test_dl_full)
        else:
            if model_name == 'SVM':
                X_test_fe = X_test_gabor  # Use Gabor features for SVM
            else:
                X_test_fe = X_test_hog  # Use HOG features for Random Forest
            y_pred_proba = model.predict_proba(X_test_fe)
        y_preds_test.append(y_pred_proba)
        # Stack predictions
    X_stack_test = np.hstack(y_preds_test)

    # Evaluate stacking hybrid model on the test set
    y_stack_pred_test = meta_learner.predict(X_stack_test)
    y_stack_pred_proba_test = meta_learner.predict_proba(X_stack_test)

    accuracy_test = accuracy_score(np.argmax(y_test_dl_full, axis=1), y_stack_pred_test)
    f1_test = f1_score(np.argmax(y_test_dl_full, axis=1), y_stack_pred_test, average='weighted')
    auc_roc_test = roc_auc_score(np.argmax(y_test_dl_full, axis=1), y_stack_pred_proba_test, multi_class='ovr')
    print("\nStacking Hybrid Model Performance on Test Set:")
    print(f"Accuracy: {accuracy_test}, F1-Score: {f1_test}, AUC-ROC: {auc_roc_test}")

elif best_hybrid_model == 'Majority Voting':
    # Get predictions from top 3 models for the test set
    y_preds_test = []
    for model_name in top_3_models:
        model = models[model_name]
        if model_name in ['VGG16', 'ResNet50', 'DenseNet', 'CNN']:
            y_pred_proba = model.predict(X_test_dl_full)
        else:
            if model_name == 'SVM':
                X_test_fe = X_test_gabor  # Use Gabor features for SVM
            else:
                X_test_fe = X_test_hog  # Use HOG features for Random Forest
            y_pred_proba = model.predict_proba(X_test_fe)
        y_preds_test.append(y_pred_proba)

    # Majority voting
    y_pred_classes_test = np.array([np.argmax(y_pred, axis=1) for y_pred in y_preds_test])
    y_majority_vote_test, _ = mode(y_pred_classes_test, axis=0)
    y_majority_vote_test = y_majority_vote_test.flatten()
  # Evaluate majority voting hybrid model on the test set
    accuracy_test = accuracy_score(np.argmax(y_test_dl_full, axis=1), y_majority_vote_test)
    f1_test = f1_score(np.argmax(y_test_dl_full, axis=1), y_majority_vote_test, average='weighted')
    auc_roc_test = roc_auc_score(np.argmax(y_test_dl_full, axis=1), np.mean(y_preds_test, axis=0), multi_class='ovr')
    print("\nMajority Voting Hybrid Model Performance on Test Set:")
    print(f"Accuracy: {accuracy_test}, F1-Score: {f1_test}, AUC-ROC: {auc_roc_test}")


In [ ]:
import joblib

# Save the best hybrid model
if best_hybrid_model == 'Weighted Averaging':
    # Save the top 3 models
    for model_name in top_3_models:
        model = models[model_name]
        if model_name in ['VGG16', 'ResNet50', 'DenseNet', 'CNN']:
            model.save(f'{model_name}_model.h5')
        else:
            joblib.dump(model, f'{model_name}_model.pkl')
elif best_hybrid_model == 'Stacking':
    # Save the meta-learner and top 3 models
    joblib.dump(meta_learner, 'stacking_meta_learner.pkl')
    for model_name in top_3_models:
        model = models[model_name]
        if model_name in ['VGG16', 'ResNet50', 'DenseNet', 'CNN']:
            model.save(f'{model_name}_model.h5')
        else:
            joblib.dump(model, f'{model_name}_model.pkl')
elif best_hybrid_model == 'Majority Voting':
    # Save the top 3 models
    for model_name in top_3_models:
        model = models[model_name]
        if model_name in ['VGG16', 'ResNet50', 'DenseNet', 'CNN']:
            model.save(f'{model_name}_model.h5')
        else:
            joblib.dump(model, f'{model_name}_model.pkl')

In [ ]:
import cv2
import numpy as np
import joblib
from tensorflow.keras.models import load_model
from scipy.stats import mode
from tkinter import Tk
from tkinter.filedialog import askopenfilename

def predict_with_hybrid_model(image_path):
    # Load the image and preprocess it
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (224, 224))
    img = img / 255.0  # Normalize
    img = np.expand_dims(img, axis=0)  # Add batch dimension

    # Load the top 3 models
    top_models = {}
    for model_name in top_3_models:
        if model_name in ['VGG16', 'ResNet50', 'DenseNet', 'CNN']:
            top_models[model_name] = load_model(f'{model_name}_model.h5')
        else:
            top_models[model_name] = joblib.load(f'{model_name}_model.pkl')

    # Get predictions from the top 3 models
    y_preds = []
    for model_name, model in top_models.items():
        if model_name in ['VGG16', 'ResNet50', 'DenseNet', 'CNN']:
            y_pred_proba = model.predict(img)
        else:
            if model_name == 'SVM':
                features = extract_gabor_features(img)
            else:
                features = extract_hog_features(img)
            y_pred_proba = model.predict_proba(features)
        y_preds.append(y_pred_proba)

    # Combine predictions based on the best hybrid model
    if best_hybrid_model == 'Weighted Averaging':
        weights = results_df.loc[top_3_models, 'Combined_Score'].values
        weights /= weights.sum()  # Normalize weights
        y_weighted_avg = np.average(y_preds, axis=0, weights=weights)
        final_pred = np.argmax(y_weighted_avg, axis=1)
    elif best_hybrid_model == 'Stacking':
        X_stack = np.hstack(y_preds)
        meta_learner = joblib.load('stacking_meta_learner.pkl')
        final_pred = meta_learner.predict(X_stack)
    elif best_hybrid_model == 'Majority Voting':
        y_pred_classes = np.array([np.argmax(y_pred, axis=1) for y_pred in y_preds])
        final_pred, _ = mode(y_pred_classes, axis=0)
        final_pred = final_pred.flatten()

    # Map prediction to class label
    class_labels = ['glioma', 'meningioma', 'notumor', 'pituitary']
    return class_labels[final_pred[0]]

# Ask the user to upload an image
Tk().withdraw()  # Hide the root window
image_path = askopenfilename(title="Select an MRI Image", filetypes=[("Image Files", "*.jpg *.jpeg *.png")])

# Make a prediction
if image_path:
    prediction = predict_with_hybrid_model(image_path)
    print("Predicted Class:", prediction)
else:
    print("No image selected.")